In [1]:
import os
import pickle
import sys

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib as mpl
import pandas as pd
import seaborn as sns


In [4]:

# Get the current working directory
current_dir = os.getcwd()

# Navigate to 'network_simulations'
parent_dir = os.path.abspath(os.path.join(current_dir, ".."))

# Navigate to the sibling directory '00_opinion_function_setup'
sibling_dir = os.path.join(parent_dir, "00_opinion_function_setup")

# Add the sibling directory to sys.path for importing
if sibling_dir not in sys.path:
    sys.path.append(sibling_dir)

# Import the module from 00_opinion_function_setup
import opinion_functions as fun  

In [5]:
# Parameters
input_folder = "input"  # Adjusted relative path
homophilyvec = [0, 0.25, 0.5, 0.75, 1]
m_values = [2, 5]
num_agents_values = [1000]  # Use the correct number of agents
num_sim = 1000


simulation_results = fun.load_simulation_results(input_folder, homophilyvec, m_values, num_agents_values, num_sim)


Files in input folder:
homophily_0.25_m_2_num_agents_10000_sim_800_maj.pkl
homophily_0.25_m_2_num_agents_10000_sim_800_min.pkl
homophily_0.25_m_2_num_agents_10000_sim_900_maj.pkl
homophily_0.25_m_2_num_agents_10000_sim_900_min.pkl
homophily_0.25_m_2_num_agents_1000_sim_1000_maj.pkl
homophily_0.25_m_2_num_agents_1000_sim_1000_min.pkl
homophily_0.25_m_5_num_agents_10000_sim_800_maj.pkl
homophily_0.25_m_5_num_agents_10000_sim_800_min.pkl
homophily_0.25_m_5_num_agents_1000_sim_1000_maj.pkl
homophily_0.25_m_5_num_agents_1000_sim_1000_min.pkl
homophily_0.5_m_2_num_agents_10000_sim_800_maj.pkl
homophily_0.5_m_2_num_agents_10000_sim_800_min.pkl
homophily_0.5_m_2_num_agents_10000_sim_900_maj.pkl
homophily_0.5_m_2_num_agents_10000_sim_900_min.pkl
homophily_0.5_m_2_num_agents_1000_sim_1000_maj.pkl
homophily_0.5_m_2_num_agents_1000_sim_1000_min.pkl
homophily_0.5_m_5_num_agents_10000_sim_800_maj.pkl
homophily_0.5_m_5_num_agents_10000_sim_800_min.pkl
homophily_0.5_m_5_num_agents_1000_sim_1000_maj.pk

In [85]:
## Specify simulation parameters to compute summary statistics for
m = 5
num_agents = 1000
minority_fraction = 0.333
majority_fraction = 1 - minority_fraction

In [86]:
# Initialize dictionaries to store results and their statistics for each key
mean_majority_opinion_stats = {}
mean_minority_opinion_stats = {}
mean_overall_opinion_stats = {}

# Iterate over homophily values
for i, homophily in enumerate(homophilyvec):
    result_key = (homophily, m, num_agents)
    if result_key not in simulation_results:
        print(f"No data for homophily={homophily}, m={m}, num_agents={num_agents}")
        continue

    # Extract simulation data
    majority_data = simulation_results[result_key]["majority"]
    minority_data = simulation_results[result_key]["minority"]

    # Compute mean opinions across simulations
    mean_majority = np.mean(majority_data, axis=1)  # Mean for each simulation
    mean_minority = np.mean(minority_data, axis=1)  # Mean for each simulation
    mean_overall = (
        minority_fraction * mean_minority + majority_fraction * mean_majority
    )

    # Compute statistics for majority opinion
    mean_majority_mean = np.mean(mean_majority)
    mean_majority_std = np.std(mean_majority)
    mean_majority_min = np.min(mean_majority)
    mean_majority_max = np.max(mean_majority)

    # Compute statistics for minority opinion
    mean_minority_mean = np.mean(mean_minority)
    mean_minority_std = np.std(mean_minority)
    mean_minority_min = np.min(mean_minority)
    mean_minority_max = np.max(mean_minority)

    # Compute statistics for overall opinion
    mean_overall_mean = np.mean(mean_overall)
    mean_overall_std = np.std(mean_overall)
    mean_overall_min = np.min(mean_overall)
    mean_overall_max = np.max(mean_overall)

    # Store the results and their statistics in dictionaries
    mean_majority_opinion_stats[result_key] = {
        "array": mean_majority,
        "mean": mean_majority_mean,
        "std": mean_majority_std,
        "min": mean_majority_min,
        "max": mean_majority_max,
    }
    mean_minority_opinion_stats[result_key] = {
        "array": mean_minority,
        "mean": mean_minority_mean,
        "std": mean_minority_std,
        "min": mean_minority_min,
        "max": mean_minority_max,
    }
    mean_overall_opinion_stats[result_key] = {
        "array": mean_overall,
        "mean": mean_overall_mean,
        "std": mean_overall_std,
        "min": mean_overall_min,
        "max": mean_overall_max,
    }


In [ ]:
# Initialize a dictionary to store the data
multiindex_data = []

# Helper function to add rows for each result_key
def add_to_multiindex_data(result_key, stats_dicts):
    row = {
        "homophily": result_key[0],
    }
    # Add statistics for each opinion type
    for opinion_type, stats_dict in stats_dicts.items():
        row[(opinion_type, "mean")] = np.round(stats_dict["mean"], 2)
        row[(opinion_type, "std")] = np.round(stats_dict["std"], 2)
        row[(opinion_type, "min")] = np.round(stats_dict["min"], 2)
        row[(opinion_type, "max")] = np.round(stats_dict["max"], 2)
    multiindex_data.append(row)

# Combine the statistics into rows
m_value = None
num_agents_value = None
for result_key in mean_majority_opinion_stats.keys():
    homophily, m_value, num_agents_value = result_key
    stats_dicts = {
        "majority": mean_majority_opinion_stats[result_key],
        "minority": mean_minority_opinion_stats[result_key],
        "overall": mean_overall_opinion_stats[result_key],
    }
    add_to_multiindex_data(result_key, stats_dicts)

# Create the DataFrame
multiindex_df = pd.DataFrame(multiindex_data)

# Set the MultiIndex columns
multiindex_df = multiindex_df.set_index(["homophily"])
multiindex_df.columns = pd.MultiIndex.from_tuples(multiindex_df.columns)

# Construct the file name
file_name = f"summary_stats/summary_statistics_m{m_value}_agents{num_agents_value}.csv"

# Save the DataFrame to the file
multiindex_df.to_csv(file_name)

print(f"DataFrame saved to {file_name}")


DataFrame saved to output/summary_statistics_m5_agents1000.csv


In [88]:
multiindex_df

majority                       minority                       \
              mean   std     min     max     mean   std    min     max   
homophily                                                                
0.00          0.31  0.31    0.00    4.95    99.83  0.30  98.80  100.00   
0.25         35.55  1.72   30.18   42.95    83.12  1.20  78.65   86.57   
0.50         66.64  1.81   60.62   72.03    66.75  1.99  59.39   72.71   
0.75         84.93  0.66   82.15   86.72    40.21  1.94  33.47   47.11   
1.00        100.00  0.00  100.00  100.00     0.00  0.00   0.00    0.00   

          overall                      
             mean   std    min    max  
homophily                              
0.00        33.45  0.28  32.90  36.60  
0.25        51.39  1.40  46.32  57.46  
0.50        66.67  1.74  60.21  71.65  
0.75        70.03  0.88  66.61  72.70  
1.00        66.70  0.00  66.70  66.70